Вот пример приложения на `wxPython`, которое использует `pandas` для чтения данных из файла Excel и отображает их в виджете `wx.grid.Grid`.

Этот пример также включает базовые возможности редактирования таблицы и сохранения изменений обратно в файл Excel.

Перед запуском кода убедитесь, что у вас установлены необходимые библиотеки.
- `wxPython` — для создания графического интерфейса.
- `pandas` — для работы с данными Excel.
- `openpyxl` — движок для чтения и записи файлов Excel (требуется для `pandas`).

### Пример кода

In [ ]:
# вариант 1

import wx
import wx.grid
import pandas as pd
from openpyxl import load_workbook

class ExcelGridWindow(wx.Frame):
    def __init__(self, parent, title, excel_file, sheet_name):
        super(ExcelGridWindow, self).__init__(parent, title=title, size=(800, 600))

        self.excel_file = excel_file  # Путь к файлу Excel
        self.sheet_name = sheet_name  # Имя листа для работы
        self.df = None  # DataFrame для хранения данных

        # Создаем панель
        panel = wx.Panel(self)
        vbox = wx.BoxSizer(wx.VERTICAL)

        # Создаем таблицу (grid)
        self.grid = wx.grid.Grid(panel)
        vbox.Add(self.grid, proportion=1, flag=wx.EXPAND | wx.ALL, border=10)

        # Кнопка для сохранения изменений
        save_button = wx.Button(panel, label="Сохранить изменения в Excel")
        save_button.Bind(wx.EVT_BUTTON, self.on_save)
        vbox.Add(save_button, flag=wx.EXPAND | wx.ALL, border=10)

        panel.SetSizer(vbox)

        # Загружаем данные из Excel
        self.load_excel_data()

        # Центрируем окно
        self.Centre()
        self.Show(True)

    def load_excel_data(self):
        try:
            # Читаем данные из Excel с помощью pandas, указывая конкретный лист
            self.df = pd.read_excel(self.excel_file, sheet_name=self.sheet_name, engine='openpyxl')

            # Создаем таблицу с нужным количеством строк и столбцов
            self.grid.CreateGrid(self.df.shape[0], self.df.shape[1])

            # Устанавливаем заголовки столбцов
            for col_idx, col_name in enumerate(self.df.columns):
                self.grid.SetColLabelValue(col_idx, str(col_name))

            # Заполняем таблицу данными
            for row_idx, row in self.df.iterrows():
                for col_idx, value in enumerate(row):
                    self.grid.SetCellValue(row_idx, col_idx, str(value))

            # Автоматически подстраиваем размеры столбцов
            self.grid.AutoSizeColumns()

        except Exception as e:
            wx.MessageBox(f"Ошибка при загрузке файла Excel: {str(e)}", "Ошибка", wx.OK | wx.ICON_ERROR)

    def on_save(self, event):
        try:
            # Считываем данные из таблицы обратно в DataFrame
            for row_idx in range(self.df.shape[0]):
                for col_idx in range(self.df.shape[1]):
                    cell_value = self.grid.GetCellValue(row_idx, col_idx)
                    # Пытаемся преобразовать значение в исходный тип данных
                    try:
                        original_type = type(self.df.iloc[row_idx, col_idx])
                        if original_type == int:
                            self.df.iloc[row_idx, col_idx] = int(cell_value)
                        elif original_type == float:
                            self.df.iloc[row_idx, col_idx] = float(cell_value)
                        else:
                            self.df.iloc[row_idx, col_idx] = cell_value
                    except ValueError:
                        self.df.iloc[row_idx, col_idx] = cell_value  # Если преобразование не удалось, оставляем как строку

            # Загружаем существующий файл Excel с помощью openpyxl
            book = load_workbook(self.excel_file, keep_vba=True)  # keep_vba=True сохраняет макросы

            # Создаем объект для записи в pandas
            writer = pd.ExcelWriter(self.excel_file, engine='openpyxl')
            writer.book = book

            # Удаляем старый лист, если он существует, чтобы заменить его новым
            if self.sheet_name in book.sheetnames:
                del book[self.sheet_name]

            # Записываем данные в указанный лист
            self.df.to_excel(writer, sheet_name=self.sheet_name, index=False)

            # Сохраняем файл
            writer.save()

            wx.MessageBox("Изменения успешно сохранены!", "Успех", wx.OK | wx.ICON_INFORMATION)

        except Exception as e:
            wx.MessageBox(f"Ошибка при сохранении файла Excel: {str(e)}", "Ошибка", wx.OK | wx.ICON_ERROR)

def main():
    app = wx.App(False)
    # Открываем диалоговое окно для выбора файла
    with wx.FileDialog(None, "Выберите файл Excel", wildcard="Excel files (*.xlsm)|*.xlsm",
                       style=wx.FD_OPEN | wx.FD_FILE_MUST_EXIST) as fileDialog:
        if fileDialog.ShowModal() == wx.ID_CANCEL:
            return  # Пользователь отменил выбор
        excel_file_path = fileDialog.GetPath()

    # Запрашиваем имя листа у пользователя (можно заменить на фиксированное имя, например, "Sheet1")
    sheet_name = wx.GetTextFromUser("Введите имя листа для работы:", "Выбор листа", "Sheet1")
    if not sheet_name:
        return  # Пользователь отменил ввод

    frame = ExcelGridWindow(None, f"Таблица Excel ({excel_file_path})", excel_file_path, sheet_name)
    app.MainLoop()

if __name__ == "__main__":
    main()

In [1]:
# Добавлена возможность работать с файлами `.xlsx`, `.xls`, и `.xlsm`, используя движки (`openpyxl` для `.xlsx` и `.xlsm`, `xlrd` для `.xls`).
# Для этого расширен фильтр в диалоговом окне выбора файла и добавлена логика выбора подходящего движка в зависимости от расширения файла.

### Обновленный код

import wx
import wx.grid
import pandas as pd
from openpyxl import load_workbook
import os

class ExcelGridWindow(wx.Frame):
    def __init__(self, parent, title, excel_file, sheet_name):
        super(ExcelGridWindow, self).__init__(parent, title=title, size=(800, 600))

        self.excel_file = excel_file  # Путь к файлу Excel
        self.sheet_name = sheet_name  # Имя листа для работы
        self.df = None  # DataFrame для хранения данных

        # Создаем панель
        panel = wx.Panel(self)
        vbox = wx.BoxSizer(wx.VERTICAL)

        # Создаем таблицу (grid)
        self.grid = wx.grid.Grid(panel)
        vbox.Add(self.grid, proportion=1, flag=wx.EXPAND | wx.ALL, border=10)

        # Кнопка для сохранения изменений
        save_button = wx.Button(panel, label="Сохранить изменения в Excel")
        save_button.Bind(wx.EVT_BUTTON, self.on_save)
        vbox.Add(save_button, flag=wx.EXPAND | wx.ALL, border=10)

        panel.SetSizer(vbox)

        # Загружаем данные из Excel
        self.load_excel_data()

        # Центрируем окно
        self.Centre()
        self.Show(True)

    def load_excel_data(self):
        try:
            # Определяем движок в зависимости от расширения файла
            file_extension = os.path.splitext(self.excel_file)[1].lower()
            if file_extension in ['.xlsx', '.xlsm']:
                engine = 'openpyxl'
            elif file_extension == '.xls':
                engine = 'xlrd'
            else:
                raise ValueError(f"Неподдерживаемый формат файла: {file_extension}")

            # Читаем данные из Excel с помощью pandas, указывая конкретный лист
            self.df = pd.read_excel(self.excel_file, sheet_name=self.sheet_name, engine=engine)

            # Создаем таблицу с нужным количеством строк и столбцов
            self.grid.CreateGrid(self.df.shape[0], self.df.shape[1])

            # Устанавливаем заголовки столбцов
            for col_idx, col_name in enumerate(self.df.columns):
                self.grid.SetColLabelValue(col_idx, str(col_name))

            # Заполняем таблицу данными
            for row_idx, row in self.df.iterrows():
                for col_idx, value in enumerate(row):
                    self.grid.SetCellValue(row_idx, col_idx, str(value))

            # Автоматически подстраиваем размеры столбцов
            self.grid.AutoSizeColumns()

        except Exception as e:
            wx.MessageBox(f"Ошибка при загрузке файла Excel: {str(e)}", "Ошибка", wx.OK | wx.ICON_ERROR)

    def on_save(self, event):
        try:
            # Считываем данные из таблицы обратно в DataFrame
            for row_idx in range(self.df.shape[0]):
                for col_idx in range(self.df.shape[1]):
                    cell_value = self.grid.GetCellValue(row_idx, col_idx)
                    # Пытаемся преобразовать значение в исходный тип данных
                    try:
                        original_type = type(self.df.iloc[row_idx, col_idx])
                        if original_type == int:
                            self.df.iloc[row_idx, col_idx] = int(cell_value)
                        elif original_type == float:
                            self.df.iloc[row_idx, col_idx] = float(cell_value)
                        else:
                            self.df.iloc[row_idx, col_idx] = cell_value
                    except ValueError:
                        self.df.iloc[row_idx, col_idx] = cell_value  # Если преобразование не удалось, оставляем как строку

            # Определяем, как сохранять файл в зависимости от его формата
            file_extension = os.path.splitext(self.excel_file)[1].lower()
            if file_extension in ['.xlsx', '.xlsm']:
                # Загружаем существующий файл Excel с помощью openpyxl
                book = load_workbook(self.excel_file, keep_vba=(file_extension == '.xlsm'))

                # Удаляем старый лист, если он существует
                if self.sheet_name in book.sheetnames:
                    del book[self.sheet_name]

                # Создаем новый лист и записываем данные
                with pd.ExcelWriter(self.excel_file, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                    writer.book = book
                    self.df.to_excel(writer, sheet_name=self.sheet_name, index=False)

                # Сохраняем файл с помощью openpyxl, чтобы сохранить макросы
                book.save(self.excel_file)

            elif file_extension == '.xls':
                # Для .xls используем движок xlwt (не поддерживает макросы и сложное форматирование)
                self.df.to_excel(self.excel_file, sheet_name=self.sheet_name, index=False, engine='xlwt')
            else:
                raise ValueError(f"Неподдерживаемый формат файла для сохранения: {file_extension}")

            wx.MessageBox("Изменения успешно сохранены!", "Успех", wx.OK | wx.ICON_INFORMATION)

        except Exception as e:
            wx.MessageBox(f"Ошибка при сохранении файла Excel: {str(e)}", "Ошибка", wx.OK | wx.ICON_ERROR)

def select_sheet(excel_file):
    """Функция для выбора листа из файла Excel."""
    try:
        # Определяем движок в зависимости от расширения файла
        file_extension = os.path.splitext(excel_file)[1].lower()
        if file_extension in ['.xlsx', '.xlsm']:
            engine = 'openpyxl'
        elif file_extension == '.xls':
            engine = 'xlrd'
        else:
            raise ValueError(f"Неподдерживаемый формат файла: {file_extension}")

        # Читаем список листов из файла
        xl = pd.ExcelFile(excel_file, engine=engine)
        sheet_names = xl.sheet_names

        # Создаем диалоговое окно для выбора листа
        with wx.SingleChoiceDialog(None, "Выберите лист для работы:", "Выбор листа",
                                   sheet_names) as dialog:
            if dialog.ShowModal() == wx.ID_OK:
                return dialog.GetStringSelection()
            else:
                return None  # Пользователь отменил выбор
    except Exception as e:
        wx.MessageBox(f"Ошибка при чтении листов из файла Excel: {str(e)}", "Ошибка", wx.OK | wx.ICON_ERROR)
        return None

def main():
    app = wx.App(False)
    # Открываем диалоговое окно для выбора файла
    with wx.FileDialog(None, "Выберите файл Excel",
                       wildcard="Excel files (*.xlsx;*.xlsm;*.xls)|*.xlsx;*.xlsm;*.xls",
                       style=wx.FD_OPEN | wx.FD_FILE_MUST_EXIST) as fileDialog:
        if fileDialog.ShowModal() == wx.ID_CANCEL:
            return  # Пользователь отменил выбор
        excel_file_path = fileDialog.GetPath()

    # Выбираем лист из файла
    sheet_name = select_sheet(excel_file_path)
    if not sheet_name:
        return  # Пользователь отменил выбор листа

    frame = ExcelGridWindow(None, f"Таблица Excel ({excel_file_path})", excel_file_path, sheet_name)
    app.MainLoop()

if __name__ == "__main__":
    main()

d:\MyRepositories\JobProjects\Общее_приложение\allapp_venv\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
d:\MyRepositories\JobProjects\Общее_приложение\allapp_venv\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


#### 1. Выбор листа из выпадающего списка
- **Функция `select_sheet`**:
  - Эта функция читает список листов из файла Excel с помощью `pd.ExcelFile` и возвращает выбранный пользователем лист.
  - Используется `wx.SingleChoiceDialog`, чтобы показать пользователю выпадающий список с именами листов. 
Пользователь может выбрать один из листов или отменить выбор.
  - Функция также учитывает формат файла, выбирая подходящий движок (`openpyxl` для `.xlsx` и `.xlsm`, `xlrd` для `.xls`).

- **Интеграция в `main`**:
  - После выбора файла вызывается функция `select_sheet`, которая возвращает имя выбранного листа. 
Если пользователь отменил выбор, приложение завершает работу.

#### 3. Поддержка других форматов
- **Расширение фильтра в `wx.FileDialog`**:
  - В диалоговом окне выбора файла теперь поддерживаются файлы `.xlsx`, `.xlsm` и `.xls`. 
Фильтр настроен как `wildcard="Excel files (*.xlsx;*.xlsm;*.xls)|*.xlsx;*.xlsm;*.xls"`.

- **Определение движка для чтения**:
  - В функции `load_excel_data` и `select_sheet` добавлена логика определения движка в зависимости от расширения файла:
    - `.xlsx` и `.xlsm` используют `openpyxl`.
    - `.xls` использует `xlrd`.

- **Сохранение изменений в зависимости от формата**:
  - В методе `on_save` добавлена логика сохранения данных в зависимости от формата файла:
    - Для `.xlsx` и `.xlsm` используется `openpyxl` с сохранением макросов (`keep_vba=True` для `.xlsm`).
    - Для `.xls` используется `xlwt` (обратите внимание, что `xlwt` не поддерживает макросы и сложное форматирование, поэтому для `.xls` файлы будут перезаписаны без сохранения таких элементов).


### Важные замечания

1. **Сохранение макросов**:
   - Для файлов `.xlsm` макросы сохраняются благодаря использованию `openpyxl` с параметром `keep_vba=True`. Однако для файлов `.xls` макросы и сложное форматирование не поддерживаются, так как используется движок `xlwt`.

2. **Типы данных**:
   - Как и в предыдущей версии кода, при сохранении данных предпринимается попытка восстановить исходный тип данных (например, числа как `int` или `float`). Если преобразование не удается, данные сохраняются как строки.

3. **Производительность**:
   - Если файл очень большой или содержит сложные макросы, сохранение изменений может занять некоторое время, особенно для `.xlsm`.

4. **Ограничения формата `.xls`**:
   - Формат `.xls` устарел и имеет ограничения (например, максимум 65 536 строк и 256 столбцов). Если вам не нужно работать с этим форматом, вы можете удалить поддержку `.xls`, оставив только `.xlsx` и `.xlsm`.

### Объяснение кода

1. **Импорт библиотек**:
   - `wx` и `wx.grid` — для создания графического интерфейса и работы с таблицами.
   - `pandas` — для чтения и записи данных в Excel.

2. **Класс `ExcelGridWindow`**:
   - Создает окно с таблицей (`wx.grid.Grid`) и кнопкой для сохранения изменений.
   - Хранит путь к файлу Excel (`self.excel_file`) и данные в виде `pandas.DataFrame` (`self.df`).

3. **Метод `load_excel_data`**:
   - Читает данные из файла Excel с помощью `pd.read_excel`.
   - Создает таблицу с нужным количеством строк и столбцов.
   - Устанавливает заголовки столбцов на основе названий колонок в DataFrame.
   - Заполняет таблицу данными, преобразуя все значения в строки (требование `wx.grid`).
   - Автоматически подстраивает размеры столбцов с помощью `AutoSizeColumns`.

4. **Метод `on_save`**:
   - Считывает данные из таблицы обратно в DataFrame.
   - Пытается сохранить исходный тип данных (например, числа как `int` или `float`), но если преобразование не удается, оставляет значение как строку.
   - Сохраняет обновленный DataFrame в файл Excel с помощью `to_excel`.
   - Показывает сообщение об успешном сохранении или об ошибке.

5. **Запуск приложения**:
   - В `if __name__ == "__main__":` указывается путь к файлу Excel (`excel_file_path`). Замените `"example.xlsx"` на путь к вашему файлу.

---

### Как использовать

1. Создайте файл Excel (например, `example.xlsx`) с данными. Пример структуры файла:

   | Имя     | Возраст | Город           |
   |---------|---------|-----------------|
   | Алексей | 25      | Москва          |
   | Елена   | 30      | Санкт-Петербург |
   | Иван    | 22      | Новосибирск     |

2. Укажите путь к этому файлу в переменной `excel_file_path` в коде.

3. Запустите программу. Вы увидите окно с таблицей, в которой можно редактировать данные.

4. После внесения изменений нажмите кнопку "Сохранить изменения в Excel", чтобы обновить исходный файл.

---

### Преимущества этого подхода

- **Простота интеграции**: Использование `pandas` позволяет легко работать с данными Excel, включая сложные структуры.
- **Функциональность**: `wx.grid.Grid` предоставляет встроенные возможности редактирования ячеек, изменения размеров столбцов и строк, а также другие функции, которые отсутствуют в `ttk.Treeview` в Tkinter.
- **Сохранение данных**: Возможность сохранять изменения обратно в файл Excel делает приложение полезным для реальных задач.

---

### Ограничения

- **Типы данных**: В примере данные в таблице отображаются как строки, и при сохранении предпринимается попытка восстановить исходный тип данных. Для более сложных случаев (например, даты или формулы) может потребоваться дополнительная обработка.
- **Производительность**: Для очень больших таблиц (тысячи строк) производительность `wx.grid` может быть ниже, чем у специализированных инструментов, таких как PyQt или PySide.

---

### Дополнительные улучшения

Если вы хотите расширить функционал приложения, вот несколько идей:

1. **Добавить кнопку для загрузки нового файла Excel**:
   - Используйте `wx.FileDialog` для выбора файла пользователем.

2. **Добавить сортировку столбцов**:
   - Реализуйте обработчик событий для заголовков столбцов, чтобы сортировать данные.

3. **Добавить поддержку форматирования**:
   - Например, выделение цветом определенных ячеек или строк на основе условий.

4. **Добавить проверку данных**:
   - Например, проверять, что в столбце "Возраст" вводятся только числа.